In [2]:
import requests
from datetime import datetime, timedelta
import ccxt
import pandas as pd
import numpy as np
import talib
import settings
import time
import joblib
from logging import getLogger,Formatter,StreamHandler,FileHandler,INFO

# モデルの読み込み
model_y_buy = joblib.load('./model_y_buy.xz')
model_y_sell = joblib.load('./model_y_sell.xz')

# ccxtのパラメータ
symbol = 'BTC/JPY'	  # 購入予定のシンボル
product_code = 'FX_BTC_JPY'
bitflyer = ccxt.bitflyer()		 # 使用する取引所を記入
bitflyer.apiKey = settings.apiKey
bitflyer.secret = settings.secret

In [32]:

parent_id = bitflyer.private_post_sendparentorder(params = {
		"order_method" : "IFD", 
		"parameters" : [
			{
				#IFD：指値で買い注文、約定したらOCO注文
				"product_code": product_code,
				"condition_type": "LIMIT",
				"side": "BUY",
				"price": 5150000,
				"size": 0.01,
			},{
				#OCO①：指値で利確注文
				"product_code": product_code, 
				"condition_type": "LIMIT", 
				"side": "SELL",
				"price": 5164000,
				"size": 0.01,
			}
		]
	}
)

print(parent_id)

{'parent_order_acceptance_id': 'JRF20220106-132327-093445'}


In [34]:
# order = bitflyer.private_get_getparentorder(
# 	params = {
# 		"parent_order_acceptance_id" : parent_id['parent_order_acceptance_id']
# 	}
# )
orders = bitflyer.fetch_open_orders(symbol = product_code, params = { "product_code" : "FX_BTC_JPY" })

print(orders)

[{'id': 'JRF20220106-131637-992535', 'clientOrderId': None, 'info': {'id': '0', 'child_order_id': 'JFX20220106-131637-400930F', 'product_code': 'FX_BTC_JPY', 'side': 'BUY', 'child_order_type': 'LIMIT', 'price': '5146611.0', 'average_price': '0.0', 'size': '0.01', 'child_order_state': 'ACTIVE', 'expire_date': '2022-02-05T13:16:37', 'child_order_date': '2022-01-06T13:16:37', 'child_order_acceptance_id': 'JRF20220106-131637-992535', 'outstanding_size': '0.01', 'cancel_size': '0.0', 'executed_size': '0.0', 'total_commission': '0.0'}, 'timestamp': 1641474997000, 'datetime': '2022-01-06T13:16:37.000Z', 'lastTradeTimestamp': None, 'status': 'open', 'symbol': 'FX_BTC_JPY', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 5146611.0, 'stopPrice': None, 'cost': 0.0, 'amount': 0.01, 'filled': 0.0, 'remaining': 0.01, 'fee': {'cost': 0.0, 'currency': None, 'rate': None}, 'average': None, 'trades': [], 'fees': [{'cost': 0.0, 'currency': None, 'rate': None}]}, {'id': 'JR

In [11]:
print(parent_id['parent_order_acceptance_id'])

JRF20220106-120213-078667


In [ ]:
# if is_buy:
# 	side = "BUY"
# 	other_side = "SELL"
# else:
# 	side = "SELL"
# 	other_side = "BUY"
	
# parent_id = bitflyer.private_post_sendparentorder(params = {
# 		"order_method" : "IFDOCO", 
# 		"minute_to_expire" : 10000, 
# 		"time_in_force" :  "GTC",
# 		"parameters" : [
# 			{
# 				#IFD：指値で買い注文、約定したらOCO注文
# 				"product_code": TRADE_PAIR,
# 				"condition_type": "LIMIT",
# 				"side": side,
# 				"price": ifd_price,
# 				"size": order_size,
# 			},{
# 				#OCO①：指値で利確注文
# 				"product_code": TRADE_PAIR, 
# 				"condition_type": "LIMIT", 
# 				"side": other_side,
# 				"price": profit_price,
# 				"size": order_size,
# 			},{
# 				#OCO②ストップ注文、トリガー価格を下回ったら成行で損切
# 				"product_code": TRADE_PAIR, 
# 				"condition_type": "STOP", 
# 				"side": other_side,
# 				"trigger_price": stop_loss_price,
# 				"size": order_size,
# 			}
# 		]
# 	}
# )